In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../../db/news_2023_2025.csv', encoding='utf-8')
df.head()

,news_id,wdate,title,article,press,url,image
0,20250523_0001,2025-05-23 19:11,[마켓인]모태펀드 존속 불확실성 해소될까…이재명 공약에 업계 주목,"2035년 종료 앞둬, 존속 공약에 기대감\n창업 초기자금 공백 완화 가능성에 업계...",이데일리,https://n.news.naver.com/mnews/article/018/000...,https://imgnews.pstatic.net/image/018/2025/05/...
1,20250523_0002,2025-05-23 18:52,"[단독] 카카오페이, 2500만 회원 쓱·스마일페이 품나…간편결제 시장 빅3 경쟁 후끈",매각가 5000억 안팎 달할듯\n결제시장 내 입지강화 포석\n카카오페이 [사진 = ...,매일경제,https://n.news.naver.com/mnews/article/009/000...,https://imgnews.pstatic.net/image/009/2025/05/...
2,20250523_0003,2025-05-23 18:38,"키스트론, 일반 청약 흥행…증거금 6조원 모았다","5월22~23일 청약 진행, 경쟁률 총2166대1\n상장 후 예상 시가총개 643억...",머니투데이,https://n.news.naver.com/mnews/article/008/000...,https://imgnews.pstatic.net/image/008/2025/05/...
3,20250523_0004,2025-05-23 18:33,"골드만삭스 차기 CEO, 이재용·김병주·이창용 만났다",[사진=연합뉴스]\n세계 2위 투자은행(IB)인 골드만삭스의 사장 겸 최고운영책임자...,매일경제,https://n.news.naver.com/mnews/article/009/000...,https://imgnews.pstatic.net/image/009/2025/05/...
4,20250523_0005,2025-05-23 18:24,흔들리는 미국…달러 패권 붕괴에 '스테이블코인' 선택한 美,/사진=게티이미지뱅크\n미국 정부가 달러의 기축통화 지위를 위협하는 금에 대응하기 ...,머니투데이,https://n.news.naver.com/mnews/article/008/000...,https://imgnews.pstatic.net/image/008/2025/05/...


In [14]:
df_1000 = df.sample(n=1000, random_state=42).reset_index(drop=True)
df_1000.head()

,news_id,wdate,title,article,press,url,image
0,20240426_0170,2024-04-26 09:51,"뮤직카우, 뉴이스트 '여보세요' 옥션 기념…'뷰민라' 티켓 이벤트",/사진제공=뮤직카우\n음악 투자 플랫폼 뮤직카우가 뉴이스트 '여보세요' 음악수익증권...,머니투데이,https://n.news.naver.com/mnews/article/008/000...,https://imgnews.pstatic.net/image/008/2024/04/...
1,20241023_0149,2024-10-23 10:46,“올 3분기 국내 오피스 4조원 거래…낮은 공실률 당분간 지속”,프라임 오피스 거래 규모\n올 3분기 국내 오피스 투자시장의 거래가 활발하게 진행된...,매일경제,https://n.news.naver.com/mnews/article/009/000...,https://imgnews.pstatic.net/image/009/2024/10/...
2,20240723_0298,2024-07-23 08:15,"비투엔, IDC건립 위한 화성 일대 토지 매입…""신성장 동력 마련""",빅데이터·인공지능(AI) 전문기업 비투엔이 신성장동력 확보에 본격 시동을 건다.\n...,아시아경제,https://n.news.naver.com/mnews/article/277/000...,https://ssl.pstatic.net/static.news/image/news...
3,20250207_0150,2025-02-07 08:01,"LG유플러스, 올해 전략적 변화 시작…실적 회복 전망-NH",[이데일리 이용성 기자] NH투자증권은 7일 LG유플러스(032640)에 대해 지난...,이데일리,https://n.news.naver.com/mnews/article/018/000...,https://imgnews.pstatic.net/image/018/2025/02/...
4,20240503_0179,2024-05-03 08:29,"하이투자증권 ""롯데칠성, 비수기 끝 성수기 기대""",[롯데칠성음료 유튜브 캡처. 재판매 및 DB 금지]\n(서울=연합뉴스) 조성흠 기자...,연합뉴스,https://n.news.naver.com/mnews/article/001/001...,https://imgnews.pstatic.net/image/001/2024/05/...


In [7]:
from pykrx.stock import get_market_ohlcv_by_date
import pandas as pd
from datetime import datetime, timedelta


def get_price_and_returns(code, base_date_str):
    base_date = datetime.strptime(base_date_str, "%Y-%m-%d")
    delta_days = [-30, -14, -7, -3, -2, -1, 0, 1, 2, 3, 7, 14, 30]

    start = (base_date + timedelta(days=min(delta_days) - 5)).strftime("%Y%m%d")
    end = (base_date + timedelta(days=max(delta_days) + 5)).strftime("%Y%m%d")
    df = get_market_ohlcv_by_date(start, end, code)
    df.index = pd.to_datetime(df.index)

    # 기준일 실제 영업일 찾기
    actual_base_date = df.index[df.index.get_indexer([base_date], method="nearest")[0]]
    base_price = df.loc[actual_base_date]["종가"]

    returns = {}
    for d in delta_days:
        if d == 0:
            continue  # 기준일은 등락률 대신 가격으로 반환
        target_date = base_date + timedelta(days=d)
        actual_target_date = df.index[
            df.index.get_indexer([target_date], method="nearest")[0]
        ]
        target_price = df.loc[actual_target_date]["종가"]
        rate = ((target_price - base_price) / base_price) * 100
        returns[f"D{d:+d}"] = round(rate, 2)

    return {"base_price": int(base_price), "returns": returns}

In [9]:
get_price_and_returns("005930", "2023-10-01")

{'base_price': 67500,
 'returns': {'D-30': np.float64(5.19),
  'D-14': np.float64(4.0),
  'D-7': np.float64(2.81),
  'D-3': np.float64(1.33),
  'D-2': np.float64(1.33),
  'D-1': np.float64(1.33),
  'D+1': np.float64(0.0),
  'D+2': np.float64(0.0),
  'D+3': np.float64(0.0),
  'D+7': np.float64(-1.63),
  'D+14': np.float64(-0.3),
  'D+30': np.float64(-0.89)}}